# Deep Convolutional Generative Adversarial Network

DCGAN is a more stabilised version of GAN. It is based on convolutional architecture. Here, *two* models are trained simultaneously by an adversarial process. A generator ("the artist") learns to create images that look real, while a discriminator ("the art critic") learns to tell real images apart from fakes. 

In [0]:
# Importing libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Utility functions

In [0]:
def preprocess_images(images):
  """
  This function will normalize the image in the range of [-1, 1]
  """

  images = (images - 127.5) / 127.5
  return images

def deprocess_images(images):
  images = (images * 127.5) + 127.5
  return images

def gen_noise(noise_shape):
  return np.random.normal(size=noise_shape)

def show_generated_images(images, save_fig: str=None):
  """
  This function will show at max 4 images in a single row.
  """

  n_images = images.shape[0]
  n_rows = int(np.ceil(n_images/4))
  for index in range(n_images):
    plt.subplot(n_rows, 4, index+1)
    denormalize_image = deprocess_images(images[index].numpy())
    denormalize_image = denormalize_image.reshape(28, 28).astype("int32")
    plt.imshow(denormalize_image)
    plt.axis("off")

  if save_fig:
    plt.savefig(save_fig)

  plt.show()

def train(generator, discriminator, real_images, noise):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)
    real_output = discriminator(real_images, training=True)
    fake_output = discriminator(generated_images, training=True)
    gen_loss = generator.generator_loss(fake_output)
    disc_loss = discriminator.discriminator_loss(real_output, fake_output)

  # calculating gradients
  gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
  disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  # adjusting weights
  generator.generator_optimizer.apply_gradients(
      zip(gen_gradients, generator.trainable_variables))
  discriminator.discriminator_optimizer.apply_gradients(
      zip(disc_gradients, discriminator.trainable_variables))
  
  return gen_loss, disc_loss

## Loading Data

In [0]:
# Loading data
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()
images = np.vstack((train_images, test_images))
images = images.reshape(images.shape[0], 28, 28, 1).astype("float32")
normalized_images = preprocess_images(images)

## Models & Losses Definition

In [0]:
class Generator(tf.keras.Model):

  def __init__(self):
    super(Generator, self).__init__()
    self.h1 = tf.keras.layers.Dense(7*7*256, use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.a1 = tf.keras.layers.LeakyReLU()
    self.reshape = tf.keras.layers.Reshape((7, 7, 256))
    self.h2 = tf.keras.layers.Conv2DTranspose(128, 5, padding="same", 
                                              use_bias=False)
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.a2 = tf.keras.layers.LeakyReLU()
    self.h3 = tf.keras.layers.Conv2DTranspose(64, 5, padding="same", strides=2, 
                                              use_bias=False)
    self.bn3 = tf.keras.layers.BatchNormalization()
    self.a3 = tf.keras.layers.LeakyReLU()
    self.gen_output = tf.keras.layers.Conv2DTranspose(1, 5, padding="same", strides=2, 
                                              use_bias=False, activation="tanh")
    self.loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    
  def call(self, inputs):
    out_h1 = self.h1(inputs)
    out_bn1 = self.bn1(out_h1)
    out_a1 = self.a1(out_bn1)
    out_reshape = self.reshape(out_a1)
    out_h2 = self.h2(out_reshape)
    out_bn2 = self.bn2(out_h2)
    out_a2 = self.a2(out_bn2)
    out_h3 = self.h3(out_a2)
    out_bn3 = self.bn3(out_h3)
    out_a3 = self.a3(out_bn3)
    return self.gen_output(out_a3)

  def generator_loss(self, fake_outputs):
    return self.loss_fn(tf.ones_like(fake_outputs), fake_outputs)

class Discriminator(tf.keras.Model):

  def __init__(self):
    super(Discriminator, self).__init__()
    self.h1 = tf.keras.layers.Conv2D(64, 5, padding="same", strides=2)
    self.a1 = tf.keras.layers.LeakyReLU()
    self.h2 = tf.keras.layers.Conv2D(128, 5, padding="same", strides=2)
    self.a2 = tf.keras.layers.LeakyReLU()
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(256)
    self.dropout = tf.keras.layers.Dropout(0.4)
    self.disc_output = tf.keras.layers.Dense(1, activation="sigmoid")
    self.loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

  def call(self, inputs):
    out_h1 = self.h1(inputs)
    out_a1 = self.a1(out_h1)
    out_h2 = self.h2(out_a1)
    out_a2 = self.a2(out_h2)
    out_flatten = self.flatten(out_a2)
    out_dense = self.dense(out_flatten)
    out_dropout = self.dropout(out_dense)
    return self.disc_output(out_dropout)

  def discriminator_loss(self, real_outputs, fake_outputs):
    fake_loss = self.loss_fn(tf.zeros_like(fake_outputs), fake_outputs)
    real_loss = self.loss_fn(tf.ones_like(real_outputs), real_outputs)
    return fake_loss + real_loss

## Training Starts

In [0]:
# Declaring hyper-parameters
EPOCHS = 50
NOISE_DIM = 100
BATCH_SIZE = 256
BUFFER_SIZE = 70000

real_images = tf.data.Dataset.from_tensor_slices(normalized_images).\
              shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Initializing models
generator = Generator()
discriminator = Discriminator()

# Training starts
for epoch in range(1, EPOCHS+1):
  for index, batch in enumerate(real_images):
    noise = gen_noise((BATCH_SIZE, NOISE_DIM))
    gen_loss, disc_loss = train(generator, discriminator, batch, noise)

    if (index+1) % 100 == 0:
      print(f"Generator & Discriminator loss after {epoch} epoch and {256*(index+1)}\
      iterations is: {gen_loss:.4f}, {disc_loss:.4f}")

  noise = gen_noise((16, NOISE_DIM))
  generated_images = generator(noise, training=False)
  show_generated_images(generated_images, f"/content/{epoch}.jpg")